In [6]:
import sys
import os
sys.path.append('../../')

%load_ext autoreload
%autoreload 2
%matplotlib inline

# os.environ['CUDA_VISIBLE_DEVICES'] = '0'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
import numpy as np

import matplotlib.image as mpimg
import matplotlib.pyplot as plt

from skimage.io import imread

from gen.load_data import load_data

In [8]:
from sklearn.utils import shuffle

train_df, valid_df, test_df = load_data('../../data')


print(train_df.head())


                                               image                   id  \
0  ../../data/Train/CameraRGB/episode_0002_000287...  episode_0002_000287   
1  ../../data/Train/CameraRGB/episode_0008_000112...  episode_0008_000112   
2                 ../../data/Train/CameraRGB/804.png                  804   
3  ../../data/Train/CameraRGB/episode_0008_000286...  episode_0008_000286   
4  ../../data/Train/CameraRGB/episode_0014_000061...  episode_0014_000061   

                                               label  
0  ../../data/Train/CameraSeg/episode_0002_000287...  
1  ../../data/Train/CameraSeg/episode_0008_000112...  
2                 ../../data/Train/CameraSeg/804.png  
3  ../../data/Train/CameraSeg/episode_0008_000286...  
4  ../../data/Train/CameraSeg/episode_0014_000061...  


In [11]:
from models.unet import model_unet_v2


model = model_unet_v2(3, image_shape=(416, 544, 3))
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 416, 544, 3)  0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 416, 544, 64) 1792        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 416, 544, 64) 256         conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 416, 544, 64) 0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [12]:
from gen.datagen import oversample_generator_from_df, balanced_generator_from_df

GPUS = 2
BATCH_SIZE = 4 * GPUS
model_dir = '../../saved_models/unet/unet_v8/'

if not os.path.exists(model_dir):
    os.mkdir(model_dir)

from gen.generators import gen_func, preprocess_label, preprocess_multi_label
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.vgg16 import VGG16, preprocess_input

rgb_gen = ImageDataGenerator()
lab_gen = ImageDataGenerator()

train_gen = gen_func(train_df, rgb_gen, lab_gen, image_size=(600, 800), target_size=(416, 544), batch_size = BATCH_SIZE, perturb=True)
valid_gen = gen_func(valid_df, rgb_gen, lab_gen, image_size=(600, 800), target_size=(416, 544), batch_size = BATCH_SIZE)

In [14]:
from keras.utils.training_utils import multi_gpu_model
from train import weighted_categorical_crossentropy
from keras.optimizers import Adam

model_gpu = multi_gpu_model(model, GPUS)

opt = Adam(1e-4)

model_gpu.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [15]:
from train import train_nn

m = train_df.shape[0]
history = train_nn(model_gpu, 
                   train_gen, 
                   valid_gen, 
                   training_size=train_df.shape[0], 
                   batch_size=BATCH_SIZE,
                   validation_size=valid_df.shape[0],
                   output_path=model_dir, 
                   epochs=100,
                   gpus = GPUS)

Instructions for updating:
Use the retry module or similar alternatives.
## Ignore next message from keras, values are replaced anyways
## Ignore next message from keras, values are replaced anyways
Epoch 1/100
Found 0 images belonging to 0 classes.
Found 0 images belonging to 0 classes.
Reinserting dataframe: 1683 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 6159 images
## Ignore next message from keras, values are replaced anyways
Found 0 images belonging to 0 classes.
Reinserting dataframe: 1683 images
Reinserting dataframe: 6159 images
384/384 [==============================] - 226s 589ms/step - loss: 0.2167 - acc: 0.9316 - val_loss: 0.3291 - val_acc: 0.8766

Epoch 00001: val_loss improved from inf to 0.32907, saving model to ../../saved_models/unet/unet_v8//model.hdf5
Epoch 2/100
384/384 [==============================] - 222s 577ms/step - loss: 0.0779 - acc: 0.9768 - val_loss: 0.2018 - val_acc:

KeyboardInterrupt: 

In [ ]:
# # summarize history for accuracy
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [11]:
model_gpu.load_weights('../../saved_models/unet/unet_v8/model.hdf5')
# model.reset_states()
model.save('../../saved_models/unet/unet_v8/model_saved.h5')